# This tool is about creating a file with all publication info off LiSyM SEEK


Source for the JSON access code: https://stackoverflow.com/questions/35120250/python-3-get-and-parse-json-api .
The rest was done, just looking at pretty-printed JSON and normal python documentation.


In [81]:
import urllib.request
import json
import datetime

# get a json of all publications
def getAllPublications():
    url = 'https://seek.lisym.org/publications.json'
    req = urllib.request.Request(url)
    r = urllib.request.urlopen(req).read()
    cont = json.loads(r.decode('utf-8'))

    return cont['data']


def getPublication(id):
    fullPublicationUrl = "https://seek.lisym.org/publications/%s.json" % id
    fullPublicationRequest = urllib.request.Request(fullPublicationUrl)
    fullPublicationText = urllib.request.urlopen(fullPublicationRequest).read()
    fullPublicationJson = json.loads(fullPublicationText.decode('utf-8'))
    return fullPublicationJson['data']

def parsePublication(data):
    attributes = data['attributes']
    authors =         attributes['authors']
    title =           attributes['title']
    journal =         attributes['journal']
    citation =        attributes['citation']
    published_date =  attributes['published_date']
    projects = data['relationships']['projects']


    return ({
          'title'  : title,
          'authors' : authors,
          'citation': citation,
          'journal': journal,
          'published_date' : published_date,
          'projects' : projects
        })


allPublicationsData = getAllPublications()

counter = 0

#print(cont)

##parsing json

publications = []

for item in allPublicationsData:
    # print(item)
    counter += 1
    id = item["id"]
    fullPublicationData = getPublication(id)
    
    publications.append(parsePublication(fullPublicationData))

# now "publications" contains a list of hashes with publication information

In [82]:
# How to match in python (I am a still a PERL person :-/) https://docs.python.org/3/library/re.html#match-objects
import re


# A method to extract the year from citation info
def getYear(s):
    if(s):
        found = re.search('20[0-9][0-9]',s)
        return int(found.group(0))
    else:
        # 0 AD (i.e. 1 BC), if the string is empty
        return 0000

# check if we extract a year and wether it is an integer
print(getYear("das Jahr 2017 war ein gutes Jahr")*2);

4034


In [111]:
# Create a "report" concerning one publication

projectCache = dict()

def getProject(id):
    ids = id
    
    if(ids in projectCache):
        # print("-----------Hit")
        return projectCache[ids]
    else:    
        fullUrl = "https://seek.lisym.org/projects/%s.json" % ids
        fullRequest = urllib.request.Request(fullUrl)
        fullText = urllib.request.urlopen(fullRequest).read()
        fullJson = json.loads(fullText.decode('utf-8'))
        projectCache[ids]=fullJson['data']
        # print("-----------MISS")
        return projectCache[ids]

def createProjectReport(projectData):
    
    
    for project in projectData['data']:
        id =  project['id']
        title = getProject(id)['attributes']['title'];
        print(title)
    

def createReport(contentHash):
    print()
    print('------------------------------------------------------------------------------')
    print(contentHash['title'])
    print(', '.join(contentHash['authors']))
    if(contentHash['citation']):
        print(contentHash['citation'])
    else:
        print("---> No citation information, please check")
    print(contentHash['journal'])
    if(contentHash['published_date']):
        print(contentHash['published_date'])
    else:
        print(getYear(contentHash['citation']))
        
    
    createProjectReport(contentHash['projects'])
print("read")

read


In [112]:
# get the date from a hash
# needed for sorting by date
def extractDate(contentHash):
    if(contentHash['published_date']):
        return datetime.datetime.strptime(contentHash['published_date'],'%Y-%m-%d')
    else:
        # now try to infer the year from citation
        if(contentHash['citation']):
            c = contentHash['citation']
            year = getYear(c)

            return datetime.datetime.strptime('%d-01-01' % year,'%Y-%m-%d')
        return datetime.datetime.strptime('1970-01-01','%Y-%m-%d')

# https://docs.python.org/3.6/tutorial/datastructures.html?highlight=data%20structures
# sorting lists
publications.sort(key=extractDate,reverse=True)

# now iterate over all publications
# and create a small report for each of them
for contentHash in publications:
    createReport(contentHash)


------------------------------------------------------------------------------
Could inherited predisposition drive non-obese fatty liver disease? Results from German tertiary referral centers
Marcin Krawczyk, Heike Bantel, Monika Rau, Jörn M Schattenberg, Frank Grünhage, Anita Pathil, Münnever Demir, Johannes Kluwe, Tobias Boettler, Susanne Weber, Andreas Geier, Frank Lammert
---> No citation information, please check
JHG
2018-02-07
Regeneration and Repair in Acute-on-Chronic Liver Failure (LiSyM-ACLF - Pillar III)

------------------------------------------------------------------------------
Effects of Gene Variants Controlling
Vitamin D Metabolism and Serum
Levels on Hepatic Steatosis
Malgorzata Jamka, Anita Arslanow, Annika Bohner, Marcin Krawczyk, Susanne Weber, Frank Grünhage, Frank Lammert, Caroline S Stokes
---> No citation information, please check
Digestion
2018-02-07
Regeneration and Repair in Acute-on-Chronic Liver Failure (LiSyM-ACLF - Pillar III)

----------------------

The Hedgehog Signalling Pathway (LiSyM-JGMMS)
LiSyM network

------------------------------------------------------------------------------
Translational learning from clinical studies predicts drug pharmacokinetics across patient populations.
M. Krauss, Ute Hofmann, Clemens Schafmayer, S. Igel, J. Schlender, C. Mueller, Mario Brosch, W. von Schoenfels, W. Erhart, A. Schuppert, M. Block, E. Schaeffeler, G. Boehmer, L. Goerlitz, J. Hoecker, J. Lippert, R. Kerb, Jochen Hampe, Lars Kuepfer, Matthias Schwab
NPJ Syst Biol Appl. 2017 Mar 28;3:11. doi: 10.1038/s41540-017-0012-5. eCollection 2017.
NPJ Syst Biol Appl
2017-06-27
Early Metabolic Injury (LiSyM-EMI - Pillar I)

------------------------------------------------------------------------------
Conditional loss of hepatocellular Hedgehog signaling in female mice leads to the persistence of hepatic steroidogenesis, androgenization and infertility
Christiane Rennert, Franziska Eplinius, Ute Hofmann, Janina Johänning, Franziska Rolfs, Wolfg

Molecular Steatosis - Imaging & Modeling (LiSyM-MSIM)

------------------------------------------------------------------------------
Full field of view time harmonic elastography of the native kidney
Tzschaetzsch H, Grossman M, Lang S, Trong M, Schultz M, Guo J, Hamm B, Braun J, Sack I.
---> No citation information, please check
Ultrasound in Medicine & Biology
0
Liver Function Diagnostics (LiSyM-LiFuDi - Pillar IV)

------------------------------------------------------------------------------
The Importance of Membrane Microdomains for Bile Salt-Dependent Biliary Lipid Secretion
Johannes Eckstein, Hergo Holzhütter, Nikolaus Berndt
---> No citation information, please check
Journal of Cell Science
0
Liver Function Diagnostics (LiSyM-LiFuDi - Pillar IV)
